# Experimentación métodos iterativos

In [37]:
import os
import csv
import numpy as np
import subprocess as sp

In [130]:
dir = "../data"

def generar_sistema(low, high, tam):
    #A = np.random.randint(low, high, (tam, tam))
    #x = np.random.randint(low, high, tam)
    A = np.array([[1, 2, -2], [1, 1, 1], [2, 2, 1]])
    x = np.array([6, -4, -3])
    b = A @ x

    if not os.path.exists(dir):
        os.makedirs(dir)

    with open(f"{dir}/sistema.txt", "w") as archivo:
        writer = csv.writer(archivo, delimiter=" ")
        writer.writerow([tam])
        for fila in A:
            writer.writerow([item for item in fila])
        for elem in x:
            writer.writerow([elem])
        for elem in b:
            writer.writerow([elem])

In [131]:
generar_sistema(-100, 100, 3)

In [156]:
def correr_algoritmo(metodo, iteraciones, error):
    proceso = sp.run(["../src/iterativo", "../data/sistema.txt", metodo, str(iteraciones), str(error)], capture_output=True, text=True)
    proceso.check_returncode()

    print(proceso.stderr)
    print(proceso.stdout)

In [169]:
correr_algoritmo("GSS", 10, 50)

El metodo diverge a la solucion

Tiempo: 0.072517
Solucion:  89.8082 -102.073  25.5296

